<a href="https://colab.research.google.com/github/Xyborg/Tiktok-Search-Autosuggest-API-Parser/blob/main/TikTokKeywordsIdeas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TikTok Search Autosuggest API Parser
#### Author: Martin Aberastegue ([@Xyborg](https://twitter.com/Xyborg))

This was made in less than an hour, just for fun and educational purposes.
The script will run through a series of loops in order to get some extra keywords, as the suggestions are limited to only 7 per keyword.

You can also generated keyword ideas by adding alphabet letters to your seed search term.

You need to enter the search term, the country, and the language code in ISO-3166 and ISO-639 respectively. For example US for USA and EN for English.

In [ ]:
%%capture
!pip install user_agent2

In [58]:
import requests
import json
from user_agent2 import (generate_user_agent)
import pandas as pd
from tqdm import tqdm
from google.colab import files
from google.colab import data_table
import string
data_table.enable_dataframe_formatter()

**Example**:

```
Keyword: iphone
Country: us
Language: en
```



In [ ]:
seedkwd = input("Keyword: ")
region = input("Country: ")
language = input("Language: ")

Keyword: iphone
Country: us
Language: en


In [ ]:
def getkwds(seed_keyword, reg, lang):
  ua = generate_user_agent(navigator="chrome")
  header = {'User-Agent': str(ua)}
  getterms_url = "https://www.tiktok.com/api/search/general/preview/?app_language=" + lang + "&keyword=" + seed_keyword + "&region=" + reg
  response = requests.get(getterms_url, headers=header)
  return response

In [60]:
srclist = []
kwdlist = []
for h in tqdm(getkwds(seedkwd, region, language).json()['sug_list'], desc='Extracting keywords'):
    srclist.append(seedkwd)
    kwdlist.append(h['content'])
    for i in getkwds(h['content'], region, language).json()['sug_list']:
      srclist.append(h['content'])
      kwdlist.append(i['content'])
      for j in getkwds(i['content'], region, language).json()['sug_list']:
        srclist.append(i['content'])
        kwdlist.append(j['content'])
        for k in getkwds(j['content'], region, language).json()['sug_list']:
          srclist.append(j['content'])
          kwdlist.append(k['content'])

Extracting keywords: 100%|██████████| 8/8 [03:15<00:00, 24.44s/it]


In [61]:
df = pd.DataFrame(None)
df['seed_keyword'] = srclist
df['related_searches'] = kwdlist

df = df.explode('related_searches').reset_index(drop=True)
print("Removing duplicates...")
df = df.drop_duplicates().reset_index(drop=True)
print(len(df), "keywords found, printing results...")
df

Removing duplicates...
2658 keywords found, printing results...


,seed_keyword,related_searches
0,iphone,iphone 14 promax
1,iphone 14 promax,iphone 14 pro max
2,iphone 14 pro max,iphone 14 pro max camera issue
3,iphone 14 pro max camera issue,iphone 14 pro max camera issue
4,iphone 14 pro max camera issue,iphone 14 pro max fake cameras
...,...,...
2653,iphone 12 mini mint green,iphone 12 mini mint green kpop
2654,iphone 12 mini mint green,iphone 12 mini mint green preppy
2655,iphone 12 mini mint green,iphone 12 mini mint green asmr
2656,iphone 12 mini mint green,iphone 12 mini mint green and nails


In [62]:
exportname = seedkwd + "-tiktok-related-kwd.csv"
df.to_csv(exportname, encoding = 'utf-8') 
files.download(exportname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

####Generate keyword ideas by adding alphabet letters to the end of the seed search term.

In [63]:
source_kws = []
final_kws = []
d = dict.fromkeys(string.ascii_lowercase, 0)
for abc in tqdm(d, desc='Extracting alphabetic related keywords'):
  kwdseed = seedkwd + " " + abc
  for iabc in getkwds(kwdseed, region, language).json()['sug_list']:
    source_kws.append(kwdseed)
    final_kws.append(iabc['content'])

dfalpha = pd.DataFrame(None)
dfalpha['seed_keyword'] = source_kws
dfalpha['related_searches'] = final_kws

dfalpha = dfalpha.explode('related_searches').reset_index(drop=True)
dfalpha = dfalpha.drop_duplicates().reset_index(drop=True)

dfalpha

Extracting alphabetic related keywords: 100%|██████████| 26/26 [00:09<00:00,  2.87it/s]


,seed_keyword,related_searches
0,iphone a,iphone aesthetic
1,iphone a,iphone aesthetic ideas
2,iphone a,iphone alarm sound
3,iphone a,iphone accessories
4,iphone a,iphone alarm
...,...,...
204,iphone z,iphone 15
205,iphone z,iphone zoom camera
206,iphone z,iphone zoom
207,iphone z,iphone zero


In [64]:
exportname = seedkwd + "-tiktok-alphabet-related-kwd.csv"
dfalpha.to_csv(exportname, encoding = 'utf-8') 
files.download(exportname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>